### QC/trimming 11/27/23 sequenced samples
- PSTR, OFAV, OANN

In [ ]:
#INSTALLATION
module load miniconda/22.11.1-1
conda create -n qc
conda activate qc
conda install -c bioconda trim-galore

In [ ]:
cd $FILEPATH 
ls *R1_001.fastq.gz -1 | sed 's/R1_001\.fastq\.gz//'
#manually put in 'sample_names'.txt because CANNOT figure out how to do it via command line...keep getting permissions error 

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=50G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 20:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/11_2023bash_scripts/slurm-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate qc

FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/Raw_sequences/SCTLD_raw/11272023'                                                                                                                                                                                                     
OUTPUT_WORKING='working/11272023'                                                                                                                                                                                                 
OUTPUT_RESULTS='working/11272023/trimmed'                                                                                                                                                                                                 
NSLOTS=5                                                                                                                                                                                                                              
                                                                                                                                                                                                                                 
#this line creates a file that gives all the filenames in the directory with the data, without the suffixes added by the sequencer                                                                                               
#ls $FILEPATH/*.fastq.gz -1 | sed 's/_R*_001.fastq.gz//' > $OUTPUT_WORKING/'11272023_samples.txt'  
#remove "undetermined" F & R seqs
#head -n -2 '11272023_samples.txt' > temp_file.txt && mv temp_file.txt '11272023_samples.txt' 


LINES=$(cat "$OUTPUT_WORKING"/'sample_names.txt')
                                                                                                                                                                                                                                 
#this loop removes adapter sequences and then quality checks the sequences and outputs a QC file                                                                                                                                                                                                                                                                                                            
for f in sample_names.txt                                                                                                                                                                                                               
do                                                                                                                                                                                                                               
    trim_galore -j $NSLOTS -q 20 --phred33 --fastqc --illumina --length 20 --paired "$f"R1_001.fastq.gz "$f"R2_001.fastq.gz;                                                                           
    fastqc -o $OUTPUT_RESULTS -t $NSLOTS "$f"R*_001.fastq.gz                                                                                                                                                        
done          

#bash script: qc
#JOB ID = 15853012


In [ ]:
# Trying chatgbt code since I can't get it

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=50G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 20:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o /project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/11_2023bash_scripts/slurm-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate qc

# Define the paths and variables
FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/Raw_sequences/SCTLD_raw/11272023'   # Replace with the actual path
OUTPUT_RESULTS='working/11272023trimmed' # Replace with the desired output directory
NSLOTS=8  # Replace with the desired number of slots

# Assuming sample_names.txt is in the same directory as the script
SAMPLE_NAMES_FILE="/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/11272023/sample_names.txt"

# Check if the file exists
if [ ! -e "$SAMPLE_NAMES_FILE" ]; then
    echo "Error: $SAMPLE_NAMES_FILE does not exist."
    exit 1
fi

# Read each line from the file and perform actions
while IFS= read -r sample_id; do
    # Form the full file names
    input_r1="$FILEPATH/${sample_id}R1_001.fastq.gz"
    input_r2="$FILEPATH/${sample_id}R2_001.fastq.gz"
    
    # Ensure the input files exist before running the tools
    if [ ! -e "$input_r1" ] || [ ! -e "$input_r2" ]; then
        echo "Error: Input files do not exist for sample $sample_id"
        continue
    fi

    # Run trim_galore
    trim_galore -j "$NSLOTS" -q 20 --phred33 --fastqc --illumina --length 20 --paired "$input_r1" "$input_r2"

    # Run fastqc
    fastqc -o "$OUTPUT_RESULTS" -t "$NSLOTS" "$input_r1" "$input_r2"

done < "$SAMPLE_NAMES_FILE"

# JOB-ID: 15853044

### Misc Samples
- PASTS samples in 04/14/23 folder 
    - (supposed to be alongside MCAV from mcav1 dir)
- also grabbing pilot seqs

In [ ]:
FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/Raw_sequences/SCTLD_raw/04142023'
OUTPUT_WORKING='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed/pilot_past'
OUTPUT_RESULTS='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed/pilot_past'
NSLOTS=5

#this line creates a file that gives all the filenames in the directory with the data, without the suffixes added by the sequencer
ls $FILEPATH/*.fastq.gz -1 | sed 's/_R*_001.fastq.gz//' |cat > $OUTPUT_WORKING/sample_names.txt


#this loop removes adapter sequences and then quality checks the sequences and outputs a QC file
LINES=$(cat "$OUTPUT_WORKING"/'sample_names.txt') 
for f in $OUTPUT_WORKING/sample_names.txt 
do 
    trim_galore -q 20 --phred33 --fastqc --illumina --length 20 --paired "$f"_R1_001.fastq "$f"_R2_001.fastq;
    fastqc -o $OUTPUT_RESULTS -t $NSLOTS "$f"_R*_001.fastq.gz
done

FILEPATH='/project/pi_sarah_gignouxwolfsohn_uml_edu/Raw_sequences/SCTLD_raw/pilot'
OUTPUT_WORKING='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed/pilot_past'
OUTPUT_RESULTS='/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/trimmed/pilot_past'


#this line creates a file that gives all the filenames in the directory with the data, without the suffixes added by the sequencer
ls $FILEPATH/*.fastq.gz -1 | sed 's/_R*_001.fastq.gz//' |print $OUTPUT_WORKING/pilot_sample_names.txt


#this loop removes adapter sequences and then quality checks the sequences and outputs a QC file
LINES=$(cat "$OUTPUT_WORKING"/'pilot_sample_names.txt') 
for f in $OUTPUT_WORKING/pilot_sample_names.txt 
do 
    trim_galore -q 20 --phred33 --fastqc --illumina --length 20 --paired "$f"_R1_001.fastq "$f"_R2_001.fastq;
    fastqc -o $OUTPUT_RESULTS -t $NSLOTS "$f"_R*_001.fastq.gz
done

### Misc code

In [ ]:
#find unique sample IDs in new raw seqs 11272023
ls | grep -i 'R1_001.fastq.gz' | grep -v '.md5' > //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/11272023/11-23-samples.txt
#ex:
ls $FILEPATH/*.fastq.gz -1 | sed 's/_R*_001.fastq.gz//' |cat > $OUTPUT_WORKING/sample_names.txt

cut -d_ -f 1-4 old_file
This simply means use _ as delimiter, and keep fields 1-4.
set2T3_7_2019_PAST_S61_R2_001.fastq.gz

ls $FILEPATH/*_PAST_*.fastq.gz -1 | sed 's/.*set2\(.*\)_PAST_.*_R.*_001\.fastq.gz/\1/' | sed 's/set2//' > $OUTPUT_WORKING/sample_names.txt

#specifically past samples in 041423 
ls $FILEPATH/*_PAST_*.fastq.gz -1 | sed 's/set2//' | cut -d_ -f 2-5,7 | cat > $OUTPUT_WORKING/sample_names.txt